# Comparing 10 most popular Partisan and Facebook pages

In [2]:
import pandas
import pandas as pd
import numpy as np
import operator
import glob
from os.path import expanduser as ospath
from urlparse import urlparse
import os
import requests
from requests_toolbelt.utils import dump
import unicodedata
import facebook
from collections import Counter
import json
import time
import facebook

In [3]:
mainstream_df = pandas.read_csv('data/mainstream_pages.csv')

In [18]:
mainstream_df

,name,id,category,verification_status,fan_count,start_type,start_date,website,link,Unnamed: 9,Unnamed: 10,Alexa Rank
0,USA TODAY,1.365236e+10,News & Media Website,blue_verified,8594579,Founded,15/9/15,http://www.usatoday.com/,https://www.facebook.com/usatoday/,NaN,NaN,482
1,Washington Post,6.250307e+09,Newspaper,blue_verified,6025913,Founded,1877,http://www.washingtonpost.com,https://www.facebook.com/washingtonpost/,NaN,NaN,175
2,The New York Times,5.281960e+09,Newspaper,blue_verified,14675258,Founded,1851,nytimes.com/chat,https://www.facebook.com/nytimes/,NaN,NaN,114
3,BBC News,2.287360e+11,Media/News Company,blue_verified,44274776,Unspecified,Nov-22,www.bbc.co.uk/news,https://www.facebook.com/bbcnews/,NaN,NaN,104
4,Forbes,3.091116e+10,Website,blue_verified,5102993,Unspecified,1917,http://www.forbes.com,https://www.facebook.com/forbes/,NaN,NaN,237
5,HuffPost,1.846876e+10,Media/News Company,blue_verified,9758390,Launched,9/5/09,www.huffpost.com,https://www.facebook.com/HuffPost/,NaN,NaN,254
6,The Guardian,1.051334e+10,Media/News Company,blue_verified,7736192,Founded,1821,http://www.theguardian.com,https://www.facebook.com/theguardian/,NaN,NaN,135
7,CNN,5.550297e+09,Media/News Company,blue_verified,28815755,Launched,1/6/01,www.cnn.com,https://www.facebook.com/cnn/,NaN,NaN,103
8,CNBC,9.721222e+10,TV Network,blue_verified,2174736,Launched,17/4/17,http://www.cnbc.com,https://www.facebook.com/cnbc/,NaN,NaN,624
9,New York Post,1.344860e+11,Newspaper,blue_verified,4054878,Unspecified,0,http://nypost.com,https://www.facebook.com/NYPost/,NaN,NaN,706


In [4]:
partisan_df = pandas.read_csv('data/Fb_pages_2.csv')

In [5]:
partisan_df.sort_values('fan_count', ascending=False).head(11)['fan_count'].mean()

7742205.363636363

In [6]:
mainstream_df.sort_values('fan_count', ascending=False)['fan_count'].mean()

13121347.0

In [7]:
partisan_df.sort_values('fan_count', ascending=False).head(30)

,name,id,category,verification_status,fan_count,start_type,start_date,website,link,mean_reactions,mean_comments,mean_shares,mean_likes,mean_loves,mean_wows,mean_hahas,mean_sads,mean_angrys,political_category
342,Fox News,1.570455e+10,Media/News Company,blue_verified,15614839,Founded,7/10/07,"http://foxnews.com/, http://insider.foxnews.com",https://www.facebook.com/FoxNews/,19609.953890,19609.953890,5534.918118,16838.592550,929.569362,208.332993,460.854554,317.919494,854.313377,right
347,NowThis,3.410000e+14,News & Media Website,blue_verified,11776185,Founded,2012,http://weibo.com/NowThis,https://www.facebook.com/NowThisNews/,16220.240650,16220.240650,11827.127380,13510.533190,1281.515952,338.161537,278.946906,463.802046,346.765147,left
286,Upworthy,3.550000e+14,Media/News Company,blue_verified,11285519,Launched,12-Mar,http://www.upworthy.com,https://www.facebook.com/Upworthy/,10321.425820,10321.425820,4287.502177,9285.294019,719.738278,90.462549,44.765641,135.773988,44.680723,left
455,ForAmerica,1.140000e+14,Nonprofit Organization,blue_verified,7919038,Launched,1/9/01,www.ForAmerica.org,https://www.facebook.com/ForAmerica/,39389.349190,39389.349190,12294.727630,38043.266700,301.254037,87.677870,319.701440,177.975556,459.401571,right
100,Occupy Democrats,3.470000e+14,Political Organization,blue_verified,6718910,Founded,21/9/21,www.occupydemocrats.com,https://www.facebook.com/OccupyDemocrats/,29131.514030,29131.514030,22710.509480,24022.561290,1215.064890,433.397100,1712.826411,424.911129,1322.481505,left
325,Faith Family America,1.850000e+14,Publisher,not_verified,5963753,Unspecified,0,http://www.faithfamilyamerica.com/,https://www.facebook.com/faithfamilyamerica/,3639.677304,3639.677304,2204.573683,3413.712095,126.166423,20.235740,17.500022,24.325223,37.679165,right
360,American News,1.790000e+14,Other,gray_verified,5565048,Unspecified,0,http://americannews.com,https://www.facebook.com/ThePatriotReview/,14886.489610,14886.489610,4494.879024,14314.091840,184.432524,31.990197,90.172704,51.829521,212.831259,right
44,Western Journalism,1.240000e+11,News & Media Website,blue_verified,5227316,Launched,25/9/25,http://www.westernjournalism.com,https://www.facebook.com/WesternJournalism/,5291.808890,5291.808890,1785.777467,5010.206077,75.476669,35.089446,54.567603,22.134214,94.298679,right
199,NRA Institute for Legislative Action,2.256108e+10,Nonprofit Organization,blue_verified,5118919,Founded,1871,http://www.NRAILA.org,https://www.facebook.com/NationalRifleAssociat...,9868.035613,9868.035613,2960.681101,9421.320113,100.805342,25.624039,58.378794,16.460138,245.390530,uncategorized
145,The Other 98%,1.150000e+14,Broadcasting & Media Production Company,blue_verified,5059627,Born,15/4/15,http://other98.com,https://www.facebook.com/TheOther98/,16855.395030,16855.395030,12862.244030,13714.705050,677.045420,220.759155,1070.292313,314.278346,858.225558,left


In [28]:
mainstream_df['fan_count']


0     8594579
1     6025913
2    14675258
3    44274776
4     5102993
5     9758390
6     7736192
7    28815755
8     2174736
9     4054878
Name: fan_count, dtype: int64

In [29]:
import numpy
numpy.corrcoef(mainstream_df['fan_count'], mainstream_df['Alexa Rank'])[0, 1]

-0.55531546251844388

In [14]:
partisan_df['mean_reactions/fan_count'] = partisan_df['mean_reactions']/ partisan_df['fan_count']

In [15]:
np.mean(partisan_df['mean_reactions/fan_count'])

0.0023706136507746843

In [10]:
len(partisan_df)

485